# Import Event Log

In [1]:
import pandas as pd
import numpy as np
import pm4py
from pm4py.objects.conversion.log import converter as log_converter

if __name__ == "__main__":
    # Read the CSV file
    dataframe_log = pd.read_csv('../../data/logs/BPIC12_Log_onlyO.csv', sep=',')

    # Drop the first column without knowing its name
    dataframe_log = dataframe_log.drop(dataframe_log.columns[0], axis=1)

    # Format the dataframe
    dataframe_log = pm4py.format_dataframe(
        dataframe_log,
        case_id='case:concept:name',
        activity_key='concept:name',
        timestamp_key='time:timestamp'
    )

    # Convert the dataframe to event log
    log = log_converter.apply(dataframe_log)

/var/folders/s_/ch_w_j2d0sqf6dbdc0_224m40000gq/T/ipykernel_4303/670060893.py:14: DeprecatedWarning: format_dataframe is deprecated as of 2.3.0 and will be removed in 3.0.0. the format_dataframe function does not need application anymore.
  dataframe_log = pm4py.format_dataframe(


# Drop unnessary columns

In [2]:
dataframe_log = dataframe_log.drop(columns=['lifecycle:transition'])

In [3]:
dataframe_log = dataframe_log.drop(columns=['case:concept:name'])

In [4]:
dataframe_log = dataframe_log.drop(columns=['@@index'])

In [5]:
dataframe_log = dataframe_log.drop(columns=['case:AMOUNT_REQ'])

In [6]:
dataframe_log = dataframe_log.drop(columns=['case:REG_DATE'])

In [7]:
from sklearn.preprocessing import StandardScaler

# Convert to datetime format
dataframe_log['time:timestamp'] = pd.to_datetime(dataframe_log['time:timestamp'])

# Calculate elapsed time since the start of each case
dataframe_log['start_time'] = dataframe_log.groupby('@@case_index')['time:timestamp'].transform('min')
dataframe_log['elapsed_time'] = (dataframe_log['time:timestamp'] - dataframe_log['start_time']).dt.total_seconds()

# Normalize the elapsed time in minutes
scaler = StandardScaler()
dataframe_log['standardized_elapsed_time'] = scaler.fit_transform(dataframe_log[['elapsed_time']])

dataframe_log = dataframe_log.drop(columns=['start_time'])
dataframe_log = dataframe_log.drop(columns=['elapsed_time'])
dataframe_log = dataframe_log.drop(columns=['time:timestamp'])

In [8]:
dataframe_log

,org:resource,concept:name,@@case_index,standardized_elapsed_time
0,10862,O_SELECTED,0,-0.620650
1,10862,O_CREATED,0,-0.620647
2,10862,O_SENT,0,-0.620647
3,11049,O_SENT_BACK,0,0.299272
4,10629,O_ACCEPTED,0,0.602251
...,...,...,...,...
31239,11003,O_SENT,5013,-0.620265
31240,10789,O_SENT_BACK,5013,0.099241
31241,10933,O_SELECTED,5014,-0.620650
31242,10933,O_CREATED,5014,-0.620648


# Insert Start & End markers

In [9]:
# Define a function to insert start and end markers
def add_markers(df):
    # Identify unique case indices
    case_indices = df['@@case_index'].unique()
    
    # Prepare a container for new DataFrame rows
    new_rows = []
    
    # Iterate over each case index to add start and end markers
    for case_index in case_indices:
        # Create a start marker row with all columns except @@case_index set to 'Start'
        start_row = {col: 'Start' if col != '@@case_index' else case_index for col in df.columns}
        
        # Create an end marker row with all columns except @@case_index set to 'End'
        end_row = {col: 'End' if col != '@@case_index' else case_index for col in df.columns}
        
        # Append start row, rows for the current case, and end row
        new_rows.append(start_row)
        new_rows.extend(df[df['@@case_index'] == case_index].to_dict('records'))
        new_rows.append(end_row)
    
    # Convert the list of rows into a DataFrame
    return pd.DataFrame(new_rows)

# Apply the function to add start and end markers to the dataframe
modified_dataframe = add_markers(dataframe_log)

# Preprocess

In [12]:
modified_dataframe['standardized_elapsed_time'] = modified_dataframe['standardized_elapsed_time'].replace({'Start': 0, 'End': 1})

In [13]:
codes, uniques = pd.factorize(modified_dataframe['org:resource'])
modified_dataframe['org:resource'] = codes

In [14]:
codes, uniques = pd.factorize(modified_dataframe['concept:name'])
modified_dataframe['concept:name'] = codes

In [15]:
modified_dataframe

,org:resource,concept:name,@@case_index,standardized_elapsed_time
0,0,0,0,0.000000
1,1,1,0,-0.620650
2,1,2,0,-0.620647
3,1,3,0,-0.620647
4,2,4,0,0.299272
...,...,...,...,...
41269,0,0,5014,0.000000
41270,47,1,5014,-0.620650
41271,47,2,5014,-0.620648
41272,47,3,5014,-0.620648


### Padding for Cases with less then 5 events

In [16]:
# Calculating the frequency of each unique value in '@@case_index'
frequency = modified_dataframe['@@case_index'].value_counts()

# Finding the minimum occurrence
min_occurrence = frequency.min()

min_occurrence

5

# Generate sliding windows

In [17]:
df_resource = modified_dataframe[['org:resource', '@@case_index']]
df_activity = modified_dataframe[['concept:name', '@@case_index']]
df_timestamp = modified_dataframe[['standardized_elapsed_time', '@@case_index']]

In [18]:
def generate_sliding_windows(df, case_id_column='@@case_index', window_size=5):
    windows = []
    targets = []
    case_indices = []

    # Iterate over each unique case
    for case_id in df[case_id_column].unique():
        # Extract the case
        case_data = df[df[case_id_column] == case_id]
        
        # Convert case_data to a NumPy array and drop the case_id_column
        case_data_array = case_data.drop(columns=[case_id_column]).to_numpy()

        # Adjusting the condition to correctly reflect window_size without needing an additional +1
        # Now it correctly considers window_size as including the target event
        if len(case_data_array) >= window_size:
            # Adjust the loop to generate sliding windows of size window_size - 1 for the inputs and use the next event as the target
            for i in range(len(case_data_array) - window_size + 1):
                # window now has window_size - 1 events
                window = case_data_array[i:i + window_size - 1]
                # The target is the event immediately following the window
                target = case_data_array[i + window_size - 1]
                windows.append(window)
                targets.append(target)
                case_indices.append(case_id)  # Store the case_id corresponding to the window

    # Convert lists to numpy arrays for easier handling and to ensure they are two-dimensional
    windows_array = np.array(windows)
    targets_array = np.array(targets)
    case_indices_array = np.array(case_indices)
    
    return windows_array, targets_array, case_indices_array

In [19]:
windows_resource, targets_resource, case_indices = generate_sliding_windows(df_resource)
windows_activity, targets_activity, case_indices = generate_sliding_windows(df_activity)
windows_timestamp, targets_timestamp, case_indices = generate_sliding_windows(df_timestamp)

# LSTM

### Architecture

In [20]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Concatenate, Embedding

# Assuming these values as placeholders, replace them with actual counts from your data
num_resources = modified_dataframe['org:resource'].nunique()
num_activities = modified_dataframe['concept:name'].nunique()

embedding_dim_resource = 50
embedding_dim_activity = 50

time_steps = 4

# Input layers
input_resource = Input(shape=(time_steps,), name='input_resource')
input_activity = Input(shape=(time_steps,), name='input_activity')
input_timestamp = Input(shape=(time_steps, 1), name='input_timestamp')

# Embedding layers
embedding_resource = Embedding(input_dim=num_resources, output_dim=embedding_dim_resource, input_length=time_steps)(input_resource)
embedding_activity = Embedding(input_dim=num_activities, output_dim=embedding_dim_activity, input_length=time_steps)(input_activity)

# LSTM layers
lstm_resource = LSTM(25, return_sequences=False)(embedding_resource)
lstm_activity = LSTM(25, return_sequences=False)(embedding_activity)
lstm_timestamp = LSTM(25, return_sequences=False)(input_timestamp)

# Concatenate outputs
concatenated = Concatenate(axis=-1)([lstm_resource, lstm_activity, lstm_timestamp])

# Output layers
output_resource = Dense(num_resources, activation='softmax', name='output_resource')(concatenated)
output_activity = Dense(num_activities, activation='softmax', name='output_activity')(concatenated)
output_timestamp = Dense(1, activation='linear', name='output_timestamp')(concatenated)

# Create and compile the model
model = Model(inputs=[input_resource, input_activity, input_timestamp], 
              outputs=[output_resource, output_activity, output_timestamp])

model.compile(optimizer='adam', 
              loss={'output_resource': 'categorical_crossentropy', 
                    'output_activity': 'categorical_crossentropy', 
                    'output_timestamp': 'mean_squared_error'},
              metrics={'output_resource': 'accuracy', 
                       'output_activity': 'accuracy', 
                       'output_timestamp': 'mean_absolute_error'})

model.summary()

2024-06-29 16:43:47.902214: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_resource (InputLayer  [(None, 4)]                  0         []                            
 )                                                                                                
                                                                                                  
 input_activity (InputLayer  [(None, 4)]                  0         []                            
 )                                                                                                
                                                                                                  
 embedding (Embedding)       (None, 4, 50)                3100      ['input_resource[0][0]']      
                                                                                              

# Data Splitting

In [21]:
from sklearn.model_selection import train_test_split

# Split the resource data
train_resource, test_resource, train_targets_resource, test_targets_resource = train_test_split(
    windows_resource, targets_resource, test_size=0.3, random_state=42)

# Split the activity data
train_activity, test_activity, train_targets_activity, test_targets_activity = train_test_split(
    windows_activity, targets_activity, test_size=0.3, random_state=42)

# Split the timestamp data
train_timestamp, test_timestamp, train_targets_timestamp, test_targets_timestamp = train_test_split(
    windows_timestamp, targets_timestamp, test_size=0.3, random_state=42)


# Training

In [22]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import LearningRateScheduler

# EarlyStopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

def cyclic_lr(epoch, lr):
    # Example function that modulates LR within a range for each epoch
    # Customize this function based on your cyclic learning rate policy
    max_lr = 0.01  # Maximum LR
    base_lr = 0.001  # Base LR
    step_size = 10  # Number of epochs for half a cycle
    cycle = np.floor(1 + epoch / (2 * step_size))
    x = np.abs(epoch / step_size - 2 * cycle + 1)
    lr = base_lr + (max_lr - base_lr) * np.maximum(0, (1 - x))
    return lr

lr_scheduler = LearningRateScheduler(cyclic_lr)

In [23]:
import numpy as np
from tensorflow.keras.utils import to_categorical


train_targets_resource_cat = to_categorical(train_targets_resource, num_classes=num_resources)
test_targets_resource_cat = to_categorical(test_targets_resource, num_classes=num_resources)

train_targets_activity_cat = to_categorical(train_targets_activity, num_classes=num_activities)
test_targets_activity_cat = to_categorical(test_targets_activity, num_classes=num_activities)


# Note: Assuming timestamp targets are continuous and don't need to be converted to categorical

history = model.fit([train_resource, train_activity, train_timestamp], 
                    [train_targets_resource_cat, train_targets_activity_cat, train_targets_timestamp],
                    epochs=25,
                    batch_size=64,
                    validation_data=([test_resource, test_activity, test_timestamp], 
                                     [test_targets_resource_cat, test_targets_activity_cat, test_targets_timestamp]),
                    verbose=1,
                    callbacks=[lr_scheduler])  # Add other callbacks as needed

Epoch 1/25
233/233 [==============================] - 10s 12ms/step - loss: 5.7891 - output_resource_loss: 3.3234 - output_activity_loss: 1.7319 - output_timestamp_loss: 0.7338 - output_resource_accuracy: 0.2276 - output_activity_accuracy: 0.3598 - output_timestamp_mean_absolute_error: 0.6250 - val_loss: 4.3681 - val_output_resource_loss: 2.7431 - val_output_activity_loss: 1.0648 - val_output_timestamp_loss: 0.5602 - val_output_resource_accuracy: 0.2809 - val_output_activity_accuracy: 0.7016 - val_output_timestamp_mean_absolute_error: 0.5329 - lr: 0.0010
Epoch 2/25
233/233 [==============================] - 1s 5ms/step - loss: 3.5686 - output_resource_loss: 2.3369 - output_activity_loss: 0.7509 - output_timestamp_loss: 0.4809 - output_resource_accuracy: 0.3989 - output_activity_accuracy: 0.7200 - output_timestamp_mean_absolute_error: 0.4857 - val_loss: 3.2041 - val_output_resource_loss: 2.0661 - val_output_activity_loss: 0.6693 - val_output_timestamp_loss: 0.4688 - val_output_resource_

In [24]:
evaluation = model.evaluate(
    [test_resource, test_activity, test_timestamp],
    [test_targets_resource_cat, test_targets_activity_cat, test_targets_timestamp]
)

print(evaluation)

199/199 [==============================] - 0s 2ms/step - loss: 2.5743 - output_resource_loss: 1.5405 - output_activity_loss: 0.6117 - output_timestamp_loss: 0.4221 - output_resource_accuracy: 0.5816 - output_activity_accuracy: 0.7351 - output_timestamp_mean_absolute_error: 0.4174
[2.574261426925659, 1.540480136871338, 0.6117017865180969, 0.42207974195480347, 0.5816182494163513, 0.7351139187812805, 0.41740697622299194]


In [25]:
# Save the model to an H5 file
model.save('dapnn_bpicO.h5')

/opt/anaconda3/lib/python3.9/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# Anomaly Score Computation

In [26]:
# Generate predictions for all inputs
predictions = model.predict([windows_resource, windows_activity, windows_timestamp])

# Extract predictions for categorical attributes (softmax probabilities)
predictions_resource = predictions[0]        # ID predictions
predictions_activity = predictions[1]  # Resource predictions

# Extract predictions for numerical attribute (timestamp)
predictions_timestamp = predictions[2] # Timestamp predictions

663/663 [==============================] - 3s 2ms/step


In [27]:
import numpy as np
from tensorflow.keras.utils import to_categorical

def compute_anomaly_scores(predictions, actuals):
    # For categorical predictions, convert actuals to one-hot for comparison
    actuals_one_hot = to_categorical(actuals, num_classes=predictions.shape[-1])
    
    max_predictions = np.max(predictions, axis=-1)
    actual_predictions = np.sum(predictions * actuals_one_hot, axis=-1)  # Extract the probability of the actual class
    
    anomaly_scores = (max_predictions - actual_predictions) / max_predictions
    
    return anomaly_scores

# Assuming targets_id, targets_resource, targets_activity, targets_role are the true values for these attributes
anomaly_scores_resource = compute_anomaly_scores(predictions_resource, targets_resource)
anomaly_scores_activity = compute_anomaly_scores(predictions_activity, targets_activity)

In [28]:
def compute_anomaly_scores_continuous(predictions, actuals, normalization_factor):
    """
    Compute anomaly scores for continuous attributes.
    
    Parameters:
    - predictions: numpy array of predicted values.
    - actuals: numpy array of actual values.
    - normalization_factor: normalization factor (e.g., standard deviation of the attribute).
    
    Returns:
    - numpy array of anomaly scores.
    """
    # Calculate absolute differences
    differences = np.abs(predictions - actuals)
    
    # Normalize the differences
    anomaly_scores = differences / normalization_factor
    
    return anomaly_scores

In [29]:
normalization_factor = np.std(targets_timestamp)       # Example normalization factor (standard deviation)
anomaly_scores_timestamp = compute_anomaly_scores_continuous(predictions_timestamp, targets_timestamp, normalization_factor)

In [30]:
import numpy as np

def classify_cases(anomaly_scores_resource, anomaly_scores_activity, anomaly_scores_timestamp, threshold=0.98):
    # Ensure all inputs are numpy arrays of the same shape
    anomaly_scores_resource = np.array(anomaly_scores_resource).flatten()
    anomaly_scores_activity = np.array(anomaly_scores_activity).flatten()
    anomaly_scores_timestamp = np.array(anomaly_scores_timestamp).flatten()

    # Check if all arrays have the same length
    if not (len(anomaly_scores_resource) == len(anomaly_scores_activity) == len(anomaly_scores_timestamp)):
        raise ValueError("All input anomaly scores must have the same length.")

    # Find the maximum anomaly score across all attributes for each case
    max_scores = np.maximum.reduce([anomaly_scores_resource, anomaly_scores_activity, anomaly_scores_timestamp])

    # Classify cases as anomalous if the maximum anomaly score exceeds the threshold
    anomalous_cases = max_scores > threshold
    
    return anomalous_cases

# Now use the anomaly scores for resource, activity, and timestamp in the classification
anomalous_cases = classify_cases(anomaly_scores_resource, anomaly_scores_activity, anomaly_scores_timestamp)

In [45]:
import numpy as np

def classify_cases(anomaly_scores_activity, threshold=0.98):
    # Ensure all inputs are numpy arrays of the same shape
    anomaly_scores_activity = np.array(anomaly_scores_activity).flatten()

    # Find the maximum anomaly score across all attributes for each case
    max_scores = np.maximum.reduce([anomaly_scores_activity])

    # Classify cases as anomalous if the maximum anomaly score exceeds the threshold
    anomalous_cases = max_scores > threshold
    
    return anomalous_cases

# Now use the anomaly scores for resource, activity, and timestamp in the classification
anomalous_cases = classify_cases(anomaly_scores_activity)

# Mapping

### True: anomaly, False: no anomaly

In [46]:
import pandas as pd

# Create a DataFrame from the case_indices_array corresponding to case_resource
mapping = pd.DataFrame({'case': case_indices})
mapping['predicted'] = anomalous_cases
mapping

,case,predicted
0,0,False
1,0,False
2,0,False
3,1,False
4,1,False
...,...,...
21209,5013,False
21210,5013,False
21211,5013,False
21212,5013,True


In [47]:
case_prediction = mapping.groupby('case')['predicted'].any()
case_prediction


case
0       False
1       False
2       False
3       False
4       False
        ...  
5010    False
5011    False
5012    False
5013     True
5014    False
Name: predicted, Length: 5015, dtype: bool

# Ground Truth

In [48]:
def generate_alignments_adjusted_tracecost_pkl(log, net, initial_marking, final_marking):
    from pm4py.algo.conformance.alignments.petri_net import algorithm as alignments
    from pm4py.algo.conformance.alignments.petri_net import variants
    from pm4py.objects.petri_net.utils import align_utils
    max_events=0
    for trace in log:
        counter=0
        for event in trace:
            counter+=1
        if counter > max_events:
            max_events=counter
    parameters={}
    parameters[alignments.Variants.VERSION_STATE_EQUATION_A_STAR.value.Parameters.PARAM_SYNC_COST_FUNCTION] = list(map(lambda i: .1*i, range(max_events*2)))
    parameters[alignments.Variants.VERSION_STATE_EQUATION_A_STAR.value.Parameters.PARAM_TRACE_COST_FUNCTION]=list(map(lambda i: align_utils.STD_MODEL_LOG_MOVE_COST-.1*i, range(max_events*2)))
    aligned_traces = alignments.apply_log(log, net, initial_marking, final_marking, variant=variants.state_equation_a_star, parameters=parameters)
    return aligned_traces

In [49]:
import pm4py
from pm4py.objects.log.importer.xes import importer as xes_importer
from pm4py.objects.bpmn.importer import importer as bpmn_importer
from pm4py.algo.conformance.alignments.petri_net import algorithm as alignments_petri

# 2. Import the given BPMN model
bpmn_graph = bpmn_importer.apply("../../data/model/Model_O.bpmn")

# 3. Convert the BPMN to a Petri net
net, initial_marking, final_marking = pm4py.convert_to_petri_net(bpmn_graph)

aligned_traces = generate_alignments_adjusted_tracecost_pkl(log, net, initial_marking, final_marking)

aligning log, completed variants ::   0%|          | 0/168 [00:00<?, ?it/s]

In [50]:
def extract_conformance_status_by_fitness(aligned_traces):
    conformance_status = []
    for alignment in aligned_traces:
        fitness = alignment['fitness']
        # If the fitness is 1.0, the trace is conforming
        if fitness == 1.0:
            conformance_status.append(1)
        else:
            conformance_status.append(0)
    return conformance_status

# Get the conformance status list from the aligned traces
conformance = extract_conformance_status_by_fitness(aligned_traces)

In [51]:
ground_truth = pd.DataFrame({'conformity': conformance})
ground_truth['predicted'] = case_prediction

# Convert False to 0 and True to 1
ground_truth['predicted'] = [int(value) for value in ground_truth['predicted']]
ground_truth['predicted'] = 1 - ground_truth['predicted']
ground_truth

,conformity,predicted
0,1,1
1,0,1
2,0,1
3,1,1
4,0,1
...,...,...
5010,0,1
5011,0,1
5012,1,1
5013,0,0


# Evaluation

In [52]:
# Calculating TP, TN, FP, FN
TP = ((ground_truth['conformity'] == 1) & (ground_truth['predicted'] == 1)).sum()
TN = ((ground_truth['conformity'] == 0) & (ground_truth['predicted'] == 0)).sum()
FP = ((ground_truth['conformity'] == 0) & (ground_truth['predicted'] == 1)).sum()
FN = ((ground_truth['conformity'] == 1) & (ground_truth['predicted'] == 0)).sum()

In [53]:
# Calculate accuracy
accuracy = (TP + TN) / (TP + TN + FP + FN)
print(f"Accuracy: {accuracy:.3f}")

Accuracy: 0.454


In [54]:
# Calculate f1

precision = TP / (TP + FP)
recall = TP / (TP + FN)

f1 = 2 * ((precision * recall) / (precision + recall))
print(f"F1: {f1:.3f}")

F1: 0.617


### Dev (Non Conform Traces)

In [55]:
# Calculate precision for Dev
precision = TN / (TN + FN)
print(f"Precision: {precision:.3f}")

Precision: 1.000


In [56]:
# Calculate recall for Dev
recall = TN / (TN + FP)
print(f"Recall: {recall:.3f}")

Recall: 0.027


### No Dev (Conform Traces)

In [57]:
# Calculate precision for No Dev
precision = TP / (TP + FP)
print(f"Precision: {precision:.3f}")

Precision: 0.446


In [58]:
# Calculate recall for No Dev
recall = TP / (TP + FN)
print(f"Recall: {recall:.3f}")

Recall: 1.000


### AUC

In [59]:
import pandas as pd
from sklearn.metrics import roc_auc_score

# Assuming ground_truth is your DataFrame
# Make sure 'conformity' contains actual labels (0 or 1)
# and 'predicted' contains predicted probabilities or scores
auc_roc = roc_auc_score(ground_truth['conformity'], ground_truth['predicted'])
auc_roc

0.513681592039801